- Title: Aggregate DataFrames in Spark
- Slug: spark-dataframe-aggregation
- Date: 2019-12-20
- Category: Programming
- Tags: programming, Scala, Spark, DataFrame, aggregation, group by
- Author: Ben Du

In [1]:
%%classpath add mvn
org.apache.spark spark-core_2.11 2.3.1
org.apache.spark spark-sql_2.11 2.3.1

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
    .builder()
    .master("local[2]")
    .appName("Spark Aggregation")
    .getOrCreate()
spark

import spark.implicits._

org.apache.spark.sql.SparkSession$implicits$@3ec6ab2

In [3]:
import org.apache.spark.sql.functions._

val df = Seq(
    ("Ben", "Du", 1),
    ("Ben", "Du", 2),
    ("Ben", "Tu", 3),
    ("Ben", "Tu", 4),
    ("Ken", "Xu", 1),
    ("Ken", "Xu", 9)
).toDF("fname", "lname", "score")
df.show

+-----+-----+-----+
|fname|lname|score|
+-----+-----+-----+
|  Ben|   Du|    1|
|  Ben|   Du|    2|
|  Ben|   Tu|    3|
|  Ben|   Tu|    4|
|  Ken|   Xu|    1|
|  Ken|   Xu|    9|
+-----+-----+-----+



null

## Aggregation Without Grouping

1. You can aggregate all values in Columns of a DataFrame.
    Just use aggregation functions in `select` without `groupBy`,
    which is very similar to SQL syntax.
    
2. Spark has no aggregation functions such as `all` and `any`. 
    However,
    they can be achieved using other aggregation functions such as `sum` and `count`.

In [4]:
df.select(
    count("fname").alias("num_first_name"),
    count("lname").alias("num_last_name"),
    sum("score").alias("sum_score")
).show

+--------------+-------------+---------+
|num_first_name|num_last_name|sum_score|
+--------------+-------------+---------+
|             6|            6|       20|
+--------------+-------------+---------+



## Aggregation Using `groupBy`

You can use position alias in group by in Spark SQL!!!

In [4]:
df.createOrReplaceTempView("people")

In [7]:
spark.sql("select fname, count(*) as n from people group by 1").show

+-----+---+
|fname|  n|
+-----+---+
|  Ben|  4|
|  Ken|  2|
+-----+---+



### sum

1. `sum` ignores `null`

2. When all values are `null`, `sum` returns `null`.

`sum` ignores `null`.

In [1]:
val df = Seq(
    ("2017-01-01", 1L),
    ("2017-01-01", 10L),
    ("2017-02-01", 2L),
    ("2017-02-01", 22L)
).toDF("date", "value").
withColumn("value", when($"value" > 20, null).otherwise($"value"))
df.show

+----------+-----+
|      date|value|
+----------+-----+
|2017-01-01|    1|
|2017-01-01|   10|
|2017-02-01|    2|
|2017-02-01| null|
+----------+-----+



df = [date: string, value: bigint]


[date: string, value: bigint]

In [16]:
df.groupBy("date").agg(sum($"value").alias("s")).show

[Stage 0:>                                                          (0 + 4) / 4]+----------+---+
|      date|  s|
+----------+---+
|2017-01-01| 11|
|2017-02-01|  2|
+----------+---+



lastException: Throwable = null


When all values are `null`, `sum` returns `null`.

In [1]:
import org.apache.spark.sql.functions._
val df = spark.read.json("../data/people.json").
    withColumn("is_null", when($"age".isNull, 1).otherwise(0))
df.show

+----+-------+-------+
| age|   name|is_null|
+----+-------+-------+
|null|Michael|      1|
|  30|   Andy|      0|
|  19| Justin|      0|
+----+-------+-------+



df = [age: bigint, name: string ... 1 more field]


[age: bigint, name: string ... 1 more field]

Specify an alias for the column after aggregation.

In [26]:
df.groupBy("is_null").agg(sum("age").alias("sage")).show

+-------+----+
|is_null|sage|
+-------+----+
|      1|null|
|      0|  49|
+-------+----+



## Group By Multiple Columns

In [7]:
df.groupBy("fname", "lname").sum().show

+-----+-----+----------+
|fname|lname|sum(score)|
+-----+-----+----------+
|  Ben|   Du|         3|
|  Ken|   Xu|        10|
|  Ben|   Tu|         7|
+-----+-----+----------+



null

In [3]:
val df = spark.read.json("../../data/people.json")
df.show

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



null

## agg

In [4]:
import org.apache.spark.sql.functions._
val df = spark.read.json("../../data/people.json").withColumn("is_null", when($"age".isNull, 1).otherwise(0))
df.show

+----+-------+-------+
| age|   name|is_null|
+----+-------+-------+
|null|Michael|      1|
|  30|   Andy|      0|
|  19| Justin|      0|
+----+-------+-------+



null

In [5]:
df.groupBy("is_null").agg(sum("age").alias("sage")).show

+-------+----+
|is_null|sage|
+-------+----+
|      1|null|
|      0|  49|
+-------+----+



null

In [6]:
df.groupBy("is_null").agg(
    sum("age").alias("sage"),
    count("*").alias("cnt")
).show

+-------+----+---+
|is_null|sage|cnt|
+-------+----+---+
|      1|null|  1|
|      0|  49|  2|
+-------+----+---+



null

## Collection

### collect_list

In [7]:
import org.apache.spark.sql.functions._

val df = Seq(
    ("Ben", 1),
    ("Ben" ,2),
    ("Ben", 3),
    ("Ken", 1),
    ("Ken", 9)
).toDF("name", "score")
df.show

+----+-----+
|name|score|
+----+-----+
| Ben|    1|
| Ben|    2|
| Ben|    3|
| Ken|    1|
| Ken|    9|
+----+-----+



null

In [13]:
val df2 = df.groupBy("name").agg(
    collect_list("score").alias("scores")
)
df2.show

+----+---------+
|name|   scores|
+----+---------+
| Ben|[1, 2, 3]|
| Ken|   [1, 9]|
+----+---------+



null

In [14]:
df2.printSchema

root
 |-- name: string (nullable = true)
 |-- scores: array (nullable = true)
 |    |-- element: integer (containsNull = true)



null

### collect_set

In [11]:
val df_copy = Seq(
    ("Ben", 1),
    ("Ben", 1),
    ("Ben" ,2),
    ("Ben", 3),
    ("Ken", 1),
    ("Ken", 9)
).toDF("name", "score")
df_copy.show

val df3 = df_copy.groupBy("name").agg(collect_list("score").alias("scores"))
df3.show()

val df4 = df_copy.groupBy("name").agg(collect_set("score").alias("scores"))
df4.show

+----+-----+
|name|score|
+----+-----+
| Ben|    1|
| Ben|    1|
| Ben|    2|
| Ben|    3|
| Ken|    1|
| Ken|    9|
+----+-----+

+----+------------+
|name|      scores|
+----+------------+
| Ben|[1, 1, 2, 3]|
| Ken|      [1, 9]|
+----+------------+

+----+---------+
|name|   scores|
+----+---------+
| Ben|[1, 2, 3]|
| Ken|   [9, 1]|
+----+---------+



null

## First/Last

## first

## last

## Grouping

### grouping

### grouping_id

In [3]:
val df = spark.read.json("../data/people.json")
df.show

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Count

### count

In [5]:
import org.apache.spark.sql.functions._
val df = spark.read.json("../../../data/people.json").withColumn("is_null", when($"age".isNull, 1).otherwise(0))
df.show

[Stage 0:>                                                          (0 + 2) / 2]+----+-------+-------+
| age|   name|is_null|
+----+-------+-------+
|null|Michael|      1|
|  30|   Andy|      0|
|  19| Justin|      0|
+----+-------+-------+



df = [age: bigint, name: string ... 1 more field]


lastException: Throwable = null


[age: bigint, name: string ... 1 more field]

In [6]:
df.groupBy("is_null").count().show()

+-------+-----+
|is_null|count|
+-------+-----+
|      1|    1|
|      0|    2|
+-------+-----+



In [7]:
df.groupBy("is_null").agg(count("*").as("total")).show

+-------+-----+
|is_null|total|
+-------+-----+
|      1|    1|
|      0|    2|
+-------+-----+



In [14]:
df.groupBy("is_null").agg(count(when($"name" === "Andy", 1).otherwise(null))).show

+-------+---------------------------------------------------+
|is_null|count(CASE WHEN (name = Andy) THEN 1 ELSE NULL END)|
+-------+---------------------------------------------------+
|      1|                                                  0|
|      0|                                                  1|
+-------+---------------------------------------------------+



In [13]:
df.groupBy("is_null").agg(sum(when($"name" === "Andy", 1).otherwise(0))).show

[Stage 41:================================>                       (44 + 8) / 75]+-------+----------------------------------------------+
|is_null|sum(CASE WHEN (name = Andy) THEN 1 ELSE 0 END)|
+-------+----------------------------------------------+
|      1|                                             0|
|      0|                                             1|
+-------+----------------------------------------------+



In [8]:
df.groupBy("is_null").agg(count("*").alias("total")).show

+-------+-----+
|is_null|total|
+-------+-----+
|      1|    1|
|      0|    2|
+-------+-----+



### countDistinct

In [16]:
df.groupBy("is_null").agg(countDistinct("is_null").alias("total")).show

+-------+-----+
|is_null|total|
+-------+-----+
|      1|    1|
|      0|    1|
+-------+-----+



### approx_count_distinct

## Sum

### sum

### sumDistinct

## Extreme Values

### max

### min

## Mean/Average

### avg

How does average behave on null values?

### mean

## Standard Deviation

### stddev

### stddev_pop

### stddev_samp

## Variance

### var_pop

### var_sample

### variance

## Correlation & Covariance

## corr 

## covar_pop

## covar_samp

## Skewness & Kurtosis

## skewness

## kurtosis

## References

https://spark.apache.org/docs/2.0.1/api/java/org/apache/spark/sql/RelationalGroupedDataset.html

https://spark.apache.org/docs/latest/sql-programming-guide.html

https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$
